##  Data Cleaning and Preprocessing Summary

In this workbook, I cleaned and preprocessed crime data recorded at the street level.  
The focus was on four police forces within London and the South East region:
**Metropolitan Police, Kent Police, Surrey Police, and Essex Police**.

The main goal of this stage was to create a reliable and consistent dataset for **Exploratory Data Analysis (EDA)**.  
To achieve this, the following steps were carried out:

- **Imported and combined** monthly CSV files from 2023–2025 for each police force.  
- **Standardised column names** for consistency and readability.  
- **Checked and removed duplicate rows** to ensure data integrity.  
- **Handled missing values** by filling categorical gaps with `"Unknown"` and retaining numeric NaNs for later spatial analysis.  
- **Removed unnecessary columns** such as `Context` to simplify the dataset.  
- **Converted month values** to proper datetime format and created new columns for `year`, `month_number`, and `month_name`.  
- **Cleaned text-based columns**  simplifying phrases like `"On or near"`.  
- **Validated coordinate ranges** to ensure longitude and latitude values were within UK boundaries.  
- **Merged** all cleaned data into one final dataset (`All_Forces_Cleaned_2023_2025.csv`) saved in the same directory as the individual files.

This cleaning and preprocessing process ensures that the dataset is **consistent, accurate, and ready** for meaningful analysis across regions and time periods.



In [34]:
# importing the main libaries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os

Here I aim to create one csv file for each police force

In [35]:
BASE = "Dataset 2023-2025" # here i am defining the base folder and is the folders name 

# Changes the forces name so that it is capitalised
FORCE_MAP = {
    "metropolitan": "Metropolitan",
    "kent": "Kent",
    "surrey": "Surrey",
    "essex": "Essex",
}

#Create a new file for the new sorted data
os.makedirs("Clean_policeforce_data", exist_ok=True)


In [36]:
files = glob.glob(f"{BASE}/20*/*.csv")   #this means “inside Dataset 2023-2025, open every subfolder starting with 20 and get every .csv file
files.sort() # sort it in chronological order

print(f"Found {len(files)} total CSV files")
print(files[:5])

Found 100 total CSV files
['Dataset 2023-2025\\2023-09\\2023-09-essex-street.csv', 'Dataset 2023-2025\\2023-09\\2023-09-kent-street.csv', 'Dataset 2023-2025\\2023-09\\2023-09-metropolitan-street.csv', 'Dataset 2023-2025\\2023-09\\2023-09-surrey-street.csv', 'Dataset 2023-2025\\2023-10\\2023-10-essex-street.csv']


### Grouping and Combining Monthly Data by Police Force

Each month's crime data was stored in separate CSV files for the Metropolitan, Kent, Surrey, and Essex police forces.  
The goal of this section was to organise and merge these monthly files into one combined dataset per police force.  

To do this:
1. I first created an empty dictionary called `force_data`, where each police force name acted as a key.
2. I then looped through all the CSV files using their file paths to:
   - Identify the corresponding **police force** from the filename.  
   - Identify the **month** from the folder name.  
   - Read the file into a DataFrame using `pandas.read_csv()`.  
   - Add two new columns:  
     - `Police_Force` — to label which force the data belongs to  
     - `Month_Folder` — to record which month the data came from
3. Each monthly DataFrame was stored inside the appropriate key in `force_data`.

After that, I looped through each police force’s list of DataFrames, combined them into one large dataset using `pd.concat()`, and saved each as a new CSV file inside the **Clean_policeforce_data** folder.


In [37]:
force_data = {name: [] for name in FORCE_MAP.values()}

for file in files:
    file_name = os.path.basename(file)
    month = os.path.basename(os.path.dirname(file))
    
    # split the file name to get the force name
    parts = file_name.replace(".csv", "").split("-")
    short_name = parts[2].lower()  # third part (metropolitan, kent, etc.)
    full_name = FORCE_MAP[short_name]
    
    # read and add force + month info
    df = pd.read_csv(file)
    df["Police_Force"] = full_name
    df["Month_Folder"] = month
    
    force_data[full_name].append(df)
    print(f"{full_name} - added {len(df)} rows from {file_name}")


Essex - added 14138 rows from 2023-09-essex-street.csv
Kent - added 16874 rows from 2023-09-kent-street.csv
Metropolitan - added 96019 rows from 2023-09-metropolitan-street.csv
Surrey - added 7824 rows from 2023-09-surrey-street.csv
Essex - added 14824 rows from 2023-10-essex-street.csv
Kent - added 16279 rows from 2023-10-kent-street.csv
Metropolitan - added 102375 rows from 2023-10-metropolitan-street.csv
Surrey - added 8002 rows from 2023-10-surrey-street.csv
Essex - added 13292 rows from 2023-11-essex-street.csv
Kent - added 14922 rows from 2023-11-kent-street.csv
Metropolitan - added 96803 rows from 2023-11-metropolitan-street.csv
Surrey - added 7296 rows from 2023-11-surrey-street.csv
Essex - added 13110 rows from 2023-12-essex-street.csv
Kent - added 14211 rows from 2023-12-kent-street.csv
Metropolitan - added 93396 rows from 2023-12-metropolitan-street.csv
Surrey - added 6875 rows from 2023-12-surrey-street.csv
Essex - added 13148 rows from 2024-01-essex-street.csv
Kent - added

In [38]:

# Loop through each police force and combine all monthly files
for force_name, data_list in force_data.items():
    if not data_list:
        print(f"No data found for {force_name}")
        continue

    # Combine all months into one DataFrame
    combined_data = pd.concat(data_list, ignore_index=True)

    
    # --- Cleaning steps before saving ---
    combined_data.drop_duplicates(inplace=True)
    combined_data.drop(['Context'], axis=1, inplace=True, errors='ignore')
    combined_data["Month"] = pd.to_datetime(combined_data["Month"], format="%Y-%m", errors="coerce")
    combined_data["Year"] = combined_data["Month"].dt.year
    combined_data["Month_number"] = combined_data["Month"].dt.month
    combined_data["Month_name"] = combined_data["Month"].dt.strftime("%b")
    combined_data["Crime type"].fillna("Unknown", inplace=True)
    combined_data["Last outcome category"].fillna("Unknown", inplace=True)
    combined_data["Location"] = (
        combined_data["Location"]
        .astype(str)
        .str.replace("On or near", "", case=False, regex=False)
        .str.strip()
    )

    # Define the file path for saving
    save_path = f"Clean_policeforce_data/{force_name.replace(' ', '_')}_all_months.csv"

    # Save the combined dataset
    combined_data.to_csv(save_path, index=False)

    # Print a confirmation message
    print(f"{force_name}: saved {len(combined_data):,} rows to {save_path}")


Metropolitan: saved 2,156,535 rows to Clean_policeforce_data/Metropolitan_all_months.csv
Kent: saved 366,684 rows to Clean_policeforce_data/Kent_all_months.csv
Surrey: saved 177,562 rows to Clean_policeforce_data/Surrey_all_months.csv
Essex: saved 326,638 rows to Clean_policeforce_data/Essex_all_months.csv


###  Initial Data Reading and Basic Checks

In this section, I began the cleaning and preprocessing stage by **reading in the crime data** for each police force and performing some **basic data checks** to understand the structure and contents of the datasets.  

Using the `head()` function, I viewed the first few rows of each dataset to get an overview of the data and confirm that it had been read correctly.  
The `info()` function was then used to inspect key details such as:
- The total number of rows and columns  
- The names and data types of each column  
- The presence of any missing values  

These initial checks provided a general understanding of the dataset’s layout and helped identify areas that might require further cleaning later on.

In [39]:
#Here I am reading the files 

met_data = pd.read_csv("clean_policeforce_data/Metropolitan_all_months.csv")
kent_data = pd.read_csv("clean_policeforce_data/Kent_all_months.csv")
surrey_data = pd.read_csv("clean_policeforce_data/Surrey_all_months.csv")
essex_data = pd.read_csv("clean_policeforce_data/Essex_all_months.csv")

In [40]:
print(met_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2156535 entries, 0 to 2156534
Data columns (total 16 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Crime ID               object 
 1   Month                  object 
 2   Reported by            object 
 3   Falls within           object 
 4   Longitude              float64
 5   Latitude               float64
 6   Location               object 
 7   LSOA code              object 
 8   LSOA name              object 
 9   Crime type             object 
 10  Last outcome category  object 
 11  Police_Force           object 
 12  Month_Folder           object 
 13  Year                   int64  
 14  Month_number           int64  
 15  Month_name             object 
dtypes: float64(2), int64(2), object(12)
memory usage: 263.2+ MB
None


In [41]:
print(met_data.head())

                                            Crime ID       Month  \
0  c0e2379bc1123d4a70f550e243f0b2ace4f5628a66f5cb...  2023-09-01   
1  06e5cb4c224db8e47a710817fdbb12ec3af70240d55c3a...  2023-09-01   
2  a751c16858b21909cdb20e928e03b7bf372be81772f334...  2023-09-01   
3  927e7e454a9e3f1701a266d1075c65f24edda2a7861f69...  2023-09-01   
4  d4b5d5f85bb768a7ddd80622f8c352f96d5680f7d442e2...  2023-09-01   

                   Reported by                 Falls within  Longitude  \
0  Metropolitan Police Service  Metropolitan Police Service   0.871752   
1  Metropolitan Police Service  Metropolitan Police Service   0.872990   
2  Metropolitan Police Service  Metropolitan Police Service   0.140127   
3  Metropolitan Police Service  Metropolitan Police Service   0.140194   
4  Metropolitan Police Service  Metropolitan Police Service   0.140576   

    Latitude         Location  LSOA code                  LSOA name  \
0  51.138669    Sydney Street  E01024002               Ashford 005B   
1  5

In [42]:
print(essex_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326638 entries, 0 to 326637
Data columns (total 16 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Crime ID               302755 non-null  object 
 1   Month                  326638 non-null  object 
 2   Reported by            326638 non-null  object 
 3   Falls within           326638 non-null  object 
 4   Longitude              321287 non-null  float64
 5   Latitude               321287 non-null  float64
 6   Location               326638 non-null  object 
 7   LSOA code              321287 non-null  object 
 8   LSOA name              321287 non-null  object 
 9   Crime type             326638 non-null  object 
 10  Last outcome category  326638 non-null  object 
 11  Police_Force           326638 non-null  object 
 12  Month_Folder           326638 non-null  object 
 13  Year                   326638 non-null  int64  
 14  Month_number           326638 non-nu

In [43]:
print(essex_data.head())

                                            Crime ID       Month  \
0  b4aac39e9627ec39b2e1c5f4da966fb0e1ff6badcb0102...  2023-09-01   
1  e6503fbcc5ed755caf7118b237ce7645102f875748f321...  2023-09-01   
2  6445517fd92c82a131b9f15a829e4f5d7f4cc79bc996b3...  2023-09-01   
3  90e6f28f8ab707367772c752efd61b5ee1dbf2319df531...  2023-09-01   
4  ff55183c5c542c4d6de87a68df0fb67e1fbfedc4cc0250...  2023-09-01   

    Reported by  Falls within  Longitude   Latitude         Location  \
0  Essex Police  Essex Police   0.435036  51.647069         The Vale   
1  Essex Police  Essex Police   0.435943  51.643479    Penwood Close   
2  Essex Police  Essex Police   0.433923  51.644628  Brookside Close   
3  Essex Police  Essex Police   0.433923  51.644628  Brookside Close   
4  Essex Police  Essex Police   0.433923  51.644628  Brookside Close   

   LSOA code      LSOA name     Crime type  \
0  E01021238  Basildon 001B       Burglary   
1  E01021238  Basildon 001B       Burglary   
2  E01021238  Basild

In [44]:
print(surrey_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177562 entries, 0 to 177561
Data columns (total 16 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Crime ID               154557 non-null  object 
 1   Month                  177562 non-null  object 
 2   Reported by            177562 non-null  object 
 3   Falls within           177562 non-null  object 
 4   Longitude              175016 non-null  float64
 5   Latitude               175016 non-null  float64
 6   Location               177562 non-null  object 
 7   LSOA code              175016 non-null  object 
 8   LSOA name              175016 non-null  object 
 9   Crime type             177562 non-null  object 
 10  Last outcome category  177562 non-null  object 
 11  Police_Force           177562 non-null  object 
 12  Month_Folder           177562 non-null  object 
 13  Year                   177562 non-null  int64  
 14  Month_number           177562 non-nu

In [45]:
print(surrey_data.head())

                                            Crime ID       Month  \
0  33a1500a09e1c89c04e79eac9d23e14f3572f717023137...  2023-09-01   
1  634eedab1983a2faf53544f14aef672193fbc013e17c3f...  2023-09-01   
2  407b2c2bb94fe41cc0186c7e2f0ebcd9e5f470aecba234...  2023-09-01   
3  a5a6615ea502fe262e338b1767b040e9aec39a7e0f2d7a...  2023-09-01   
4                                                NaN  2023-09-01   

     Reported by   Falls within  Longitude   Latitude      Location  \
0  Surrey Police  Surrey Police  -0.101292  51.514770    Amen Court   
1  Surrey Police  Surrey Police  -0.101292  51.514770    Amen Court   
2  Surrey Police  Surrey Police  -0.101292  51.514770    Amen Court   
3  Surrey Police  Surrey Police  -0.101292  51.514770    Amen Court   
4  Surrey Police  Surrey Police  -0.197082  51.160809  Parking Area   

   LSOA code            LSOA name                    Crime type  \
0  E01032739  City of London 001F                         Drugs   
1  E01032739  City of London 0

In [46]:
print(kent_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366684 entries, 0 to 366683
Data columns (total 16 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Crime ID               324391 non-null  object 
 1   Month                  366684 non-null  object 
 2   Reported by            366684 non-null  object 
 3   Falls within           366684 non-null  object 
 4   Longitude              362299 non-null  float64
 5   Latitude               362299 non-null  float64
 6   Location               366684 non-null  object 
 7   LSOA code              362299 non-null  object 
 8   LSOA name              362299 non-null  object 
 9   Crime type             366684 non-null  object 
 10  Last outcome category  366684 non-null  object 
 11  Police_Force           366684 non-null  object 
 12  Month_Folder           366684 non-null  object 
 13  Year                   366684 non-null  int64  
 14  Month_number           366684 non-nu

In [47]:
print(kent_data.head())

                                            Crime ID       Month  Reported by  \
0  2b039a539f988ccef75fd48e7831ff5d378a8dc67ed4e7...  2023-09-01  Kent Police   
1  83b1590794616be4f2f32662934574bc1051f6b4d561d1...  2023-09-01  Kent Police   
2  96a16909d14a95c1d1c2805adba0d51d1493ce4aff25c8...  2023-09-01  Kent Police   
3  ab1f3788ae7878f2a9a6de6aec52fb595f0734373e0d2b...  2023-09-01  Kent Police   
4  54f777cecd830071d00c549f5cd0b9b1e7004fc099ba51...  2023-09-01  Kent Police   

  Falls within  Longitude   Latitude          Location  LSOA code  \
0  Kent Police   0.970011  51.253133        Cobbs Hill  E01023987   
1  Kent Police   0.967039  51.244825  Felborough Close  E01023987   
2  Kent Police   0.965680  51.243969      Herons Close  E01023987   
3  Kent Police   0.970011  51.253133        Cobbs Hill  E01023987   
4  Kent Police   0.970011  51.253133        Cobbs Hill  E01023987   

      LSOA name                    Crime type  \
0  Ashford 001B  Violence and sexual offences   


This section is dedicated to cleaning each csv file


### Checking for Duplicate Rows

Before analysing the data, I checked for duplicate rows to ensure that no records were repeated.  
Duplicate entries can distort results by inflating crime counts or affecting trend accuracy, so it’s important to confirm that each record is unique.


In [48]:
# starting with droping off all duplicates in each df
# started of by comparing the count of each df
print("Metropolitan:", len(met_data), "rows")
print("Kent:", len(kent_data), "rows")
print("Surrey:", len(surrey_data), "rows")
print("Essex:", len(essex_data), "rows")


Metropolitan: 2156535 rows
Kent: 366684 rows
Surrey: 177562 rows
Essex: 326638 rows


In [49]:
# droping the duplicates if there are any
met_data.drop_duplicates(inplace=True)
kent_data.drop_duplicates(inplace=True)
surrey_data.drop_duplicates(inplace=True)
essex_data.drop_duplicates(inplace=True)


In [50]:
print("Metropolitan:", len(met_data), "rows")
print("Kent:", len(kent_data), "rows")
print("Surrey:", len(surrey_data), "rows")
print("Essex:", len(essex_data), "rows")
# this show that there are no duplicates found in each dataset 


Metropolitan: 2156535 rows
Kent: 366684 rows
Surrey: 177562 rows
Essex: 326638 rows


### Converting the Month Column to a Date Format

The `month` column was originally stored as text in the format `YYYY-MM` (for example, `2023-09`), which pandas recognised as an **object** type.  
To make it suitable for time-based analysis, I converted this column into a proper **datetime** format using the `pd.to_datetime()` function.

This step allows for easier extraction of time-based details such as the **year**, **month number**, and **month name**.  
These will later be used to analyse crime trends across different months and years.


In [51]:
#currently the month colunm data type is a object and need to be converted to datetime type
datasets = [met_data, kent_data, surrey_data, essex_data]


for df in datasets:
    df["Month"] = pd.to_datetime(df["Month"], format="%Y-%m", errors="coerce")
    df["Year"] = df["Month"].dt.year
    df["Month_number"] = df["Month"].dt.month
    df["Month_name"] = df["Month"].dt.strftime("%b")


In [52]:
# Double checking the data
print(kent_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366684 entries, 0 to 366683
Data columns (total 16 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Crime ID               324391 non-null  object        
 1   Month                  0 non-null       datetime64[ns]
 2   Reported by            366684 non-null  object        
 3   Falls within           366684 non-null  object        
 4   Longitude              362299 non-null  float64       
 5   Latitude               362299 non-null  float64       
 6   Location               366684 non-null  object        
 7   LSOA code              362299 non-null  object        
 8   LSOA name              362299 non-null  object        
 9   Crime type             366684 non-null  object        
 10  Last outcome category  366684 non-null  object        
 11  Police_Force           366684 non-null  object        
 12  Month_Folder           366684 non-null  obje

In [53]:
print(essex_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326638 entries, 0 to 326637
Data columns (total 16 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Crime ID               302755 non-null  object        
 1   Month                  0 non-null       datetime64[ns]
 2   Reported by            326638 non-null  object        
 3   Falls within           326638 non-null  object        
 4   Longitude              321287 non-null  float64       
 5   Latitude               321287 non-null  float64       
 6   Location               326638 non-null  object        
 7   LSOA code              321287 non-null  object        
 8   LSOA name              321287 non-null  object        
 9   Crime type             326638 non-null  object        
 10  Last outcome category  326638 non-null  object        
 11  Police_Force           326638 non-null  object        
 12  Month_Folder           326638 non-null  obje

In [54]:
print(met_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2156535 entries, 0 to 2156534
Data columns (total 16 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   Crime ID               object        
 1   Month                  datetime64[ns]
 2   Reported by            object        
 3   Falls within           object        
 4   Longitude              float64       
 5   Latitude               float64       
 6   Location               object        
 7   LSOA code              object        
 8   LSOA name              object        
 9   Crime type             object        
 10  Last outcome category  object        
 11  Police_Force           object        
 12  Month_Folder           object        
 13  Year                   float64       
 14  Month_number           float64       
 15  Month_name             object        
dtypes: datetime64[ns](1), float64(4), object(11)
memory usage: 263.2+ MB
None


In [55]:
print(surrey_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177562 entries, 0 to 177561
Data columns (total 16 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Crime ID               154557 non-null  object        
 1   Month                  0 non-null       datetime64[ns]
 2   Reported by            177562 non-null  object        
 3   Falls within           177562 non-null  object        
 4   Longitude              175016 non-null  float64       
 5   Latitude               175016 non-null  float64       
 6   Location               177562 non-null  object        
 7   LSOA code              175016 non-null  object        
 8   LSOA name              175016 non-null  object        
 9   Crime type             177562 non-null  object        
 10  Last outcome category  177562 non-null  object        
 11  Police_Force           177562 non-null  object        
 12  Month_Folder           177562 non-null  obje

###  Removing the 'Context' Column

When inspecting the dataset using the `info()` function, the `Context` column showed **0 non-null values** across all records.  
This indicates that the column was completely empty and contained no useful data.

Since the column added no analytical value and was entirely composed of missing values, it was removed from all datasets to simplify further processing.


In [56]:
#Here I am double checking the met police force as it is not showing me any information about 'null' values
print(met_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2156535 entries, 0 to 2156534
Data columns (total 16 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   Crime ID               object        
 1   Month                  datetime64[ns]
 2   Reported by            object        
 3   Falls within           object        
 4   Longitude              float64       
 5   Latitude               float64       
 6   Location               object        
 7   LSOA code              object        
 8   LSOA name              object        
 9   Crime type             object        
 10  Last outcome category  object        
 11  Police_Force           object        
 12  Month_Folder           object        
 13  Year                   float64       
 14  Month_number           float64       
 15  Month_name             object        
dtypes: datetime64[ns](1), float64(4), object(11)
memory usage: 263.2+ MB
None


In [58]:
#missing = met_data["Context"].isna().sum()
#total = len(df)
#percent_missing = (missing / total) * 100
#print(missing)


In [59]:
#This justifies droping the columns 'Context' in all the datasets 

for df in datasets:
    df.drop(['Context'], axis=1,inplace=True, errors='ignore')

print(met_data.columns)
print(surrey_data.columns)
print(kent_data.columns)
print(essex_data.columns)


Index(['Crime ID', 'Month', 'Reported by', 'Falls within', 'Longitude',
       'Latitude', 'Location', 'LSOA code', 'LSOA name', 'Crime type',
       'Last outcome category', 'Police_Force', 'Month_Folder', 'Year',
       'Month_number', 'Month_name'],
      dtype='object')
Index(['Crime ID', 'Month', 'Reported by', 'Falls within', 'Longitude',
       'Latitude', 'Location', 'LSOA code', 'LSOA name', 'Crime type',
       'Last outcome category', 'Police_Force', 'Month_Folder', 'Year',
       'Month_number', 'Month_name'],
      dtype='object')
Index(['Crime ID', 'Month', 'Reported by', 'Falls within', 'Longitude',
       'Latitude', 'Location', 'LSOA code', 'LSOA name', 'Crime type',
       'Last outcome category', 'Police_Force', 'Month_Folder', 'Year',
       'Month_number', 'Month_name'],
      dtype='object')
Index(['Crime ID', 'Month', 'Reported by', 'Falls within', 'Longitude',
       'Latitude', 'Location', 'LSOA code', 'LSOA name', 'Crime type',
       'Last outcome category', 

### Handling Missing Location Data

Some records in the dataset were missing geographical coordinates (`longitude` and `latitude`).  
Since these rows still contain useful information (such as crime type, month, and outcome),  
I decided **not to remove them** from the dataset.

Instead:
- Missing text-based values (e.g., in `crime_type` and `last_outcome_category`) were replaced with **"Unknown"**.
- Missing numeric values in coordinates were left as **NaN**, and will simply be **ignored** in visualisations or analyses that require map data.


In [60]:
# Fill missing text values but keep numeric NaN values (like longitude/latitude)
#datasets = [met_data, kent_data, surrey_data, essex_data]
for df in datasets:
    df["Crime type"].fillna("Unknown", inplace=True)
    df["Last outcome category"].fillna("Unknown", inplace=True)

    

In [61]:
#double cheaking that theere is no more null values 
for name, df in zip(["Metropolitan", "Kent", "Surrey", "Essex"], datasets):
    print(name)
    print(df[["Crime type", "Last outcome category"]].isna().sum())


Metropolitan
Crime type               0
Last outcome category    0
dtype: int64
Kent
Crime type               0
Last outcome category    0
dtype: int64
Surrey
Crime type               0
Last outcome category    0
dtype: int64
Essex
Crime type               0
Last outcome category    0
dtype: int64


In [62]:
#here I created a table to show how many missing values there are in the each csv when it came to longitude and latitude 
summary = []

for name, df in zip(["Metropolitan", "Kent", "Surrey", "Essex"], datasets):
    summary.append({
        "Force": name,
        "Missing Longitude": df["Longitude"].isna().sum(),
        "Missing Latitude": df["Latitude"].isna().sum(),
    })

pd.DataFrame(summary)

,Force,Missing Longitude,Missing Latitude
0,Metropolitan,10188,10188
1,Kent,4385,4385
2,Surrey,2546,2546
3,Essex,5351,5351


### Cleaning Text Columns  
This involved:
- Removing repetitive prefixes like `"On or near"` from the `Location` column  

In [63]:
#Clean up text columns for consistency
for df in datasets:
    # Here I am removing 'On or near' to simplify the location names
    df["Location"] = df["Location"].astype(str).str.replace("On or near", "", case=False, regex=False).str.strip()


In [64]:
#Here I am confirming the prior code worked
print(df["Location"].unique()[:10])

['The Vale' 'Penwood Close' 'Brookside Close' 'Glanmire' 'Mount View'
 'Cranmer Close' 'Trafalgar Way' 'Orchard Avenue' 'Tavistock Drive'
 'Wesley Gardens']


In [65]:
# Save each cleaned dataset so changes persist for EDA
met_data.to_csv("Clean_policeforce_data/Metropolitan_all_months.csv", index=False)
kent_data.to_csv("Clean_policeforce_data/Kent_all_months.csv", index=False)
surrey_data.to_csv("Clean_policeforce_data/Surrey_all_months.csv", index=False)
essex_data.to_csv("Clean_policeforce_data/Essex_all_months.csv", index=False)


### Combining All Regional Datasets

After cleaning each regional dataset (Metropolitan, Kent, Surrey, and Essex),  
they were combined into a single DataFrame to allow for unified analysis across all police forces.


In [66]:

all_forces = pd.concat(datasets, ignore_index=True)

In [67]:
all_forces.to_csv("Clean_policeforce_data/All_Forces_Cleaned_2023_2025.csv", index=False)

In [68]:
print(met_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2156535 entries, 0 to 2156534
Data columns (total 16 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   Crime ID               object        
 1   Month                  datetime64[ns]
 2   Reported by            object        
 3   Falls within           object        
 4   Longitude              float64       
 5   Latitude               float64       
 6   Location               object        
 7   LSOA code              object        
 8   LSOA name              object        
 9   Crime type             object        
 10  Last outcome category  object        
 11  Police_Force           object        
 12  Month_Folder           object        
 13  Year                   float64       
 14  Month_number           float64       
 15  Month_name             object        
dtypes: datetime64[ns](1), float64(4), object(11)
memory usage: 263.2+ MB
None
